In [0]:
%sql
CREATE OR REPLACE TABLE geospatial.greenspaces.top_parks_gold AS ( 
WITH tmp AS (
SELECT 
gs.id,
gs.name AS park_name,
gs.functions,
gs.num_functions,
gs.area_category,
sum(rn.degree) AS accessibility_degree,
cc.name AS county,
cc.number_of_locations,
row_number() OVER (PARTITION BY cc.name ORDER BY gs.area_category DESC, gs.num_functions DESC, sum(degree) DESC) AS park_rank,
ST_AsEWKB(gs.geometry) AS geometry
FROM geospatial.greenspaces.greenspace_site_silver gs
LEFT JOIN geospatial.greenspaces.access_point_silver ga
ON gs.id = ga.ref_to_greenspace_site
LEFT JOIN geospatial.networks.road_node_silver rn
ON ga.nearest_road_node_fid = rn.fid
LEFT JOIN geospatial.lookups.boundary_line_ceremonial_counties_silver cc
ON gs.administrative_fid = cc.fid
GROUP BY gs.name, gs.functions, gs.num_functions, gs.area_category, cc.number_of_locations, cc.name, gs.geometry, gs.id, cc.name)
SELECT * FROM tmp WHERE park_rank <= number_of_locations);

 

In [0]:
%sql
CREATE OR REPLACE TABLE geospatial.greenspaces.top_entrances_gold AS (
WITH A AS (
SELECT 
ga.id,
ga.ref_to_greenspace_site AS park_id,
gs.park_rank,
ga.access_type,
rn.degree,
ga.distance_to_road_node,
row_number() OVER (PARTITION BY gs.id ORDER BY rn.degree DESC, ga.distance_to_road_node ASC) AS entry_rank,
ST_AsEWKB(ga.geometry) AS geometry,
ST_X(ST_Transform(ga.geometry,'epsg:27700','epsg:4326')) AS longitude,
ST_Y(ST_Transform(ga.geometry,'epsg:27700','epsg:4326')) AS latitude
FROM geospatial.greenspaces.top_parks_gold gs
LEFT JOIN geospatial.greenspaces.access_point_silver ga
ON gs.id = ga.ref_to_greenspace_site
LEFT JOIN geospatial.networks.road_node_silver rn
ON ga.nearest_road_node_fid = rn.fid) 
SELECT * FROM A WHERE entry_rank = 1
);